<a href="https://colab.research.google.com/github/Interpause/pseudo-text/blob/master/true_news_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install newspaper3k
import newspaper as news
from newspaper import Article, Source, Config
from newspaper.mthreading import NewsPool

In [0]:
from google.colab import drive #drive.flush_and_unmount()
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/CSIT Internship/code')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Data cleaning 2.0

In [0]:
df = pd.read_pickle('./raw/myNews3.pkl')

In [0]:
import unicodedata
#normally also helps remove foreign languages
removables = ['AD']
def clean(x):
    x.title = ' '.join(w for w in unicodedata.normalize('NFKD',x.title.strip()).encode('ascii','ignore').decode('utf8','ignore').split() if w not in removables)
    x.text = ' '.join(w for w in unicodedata.normalize('NFKD',x.text.strip()).encode('ascii','ignore').decode('utf8','ignore').split() if w not in removables)
    if len(x.title) < 1 or len(x.text.split()) < config.MIN_WORD_COUNT:
        x.title = 'None'
        x.text = 'None'
    return x

In [0]:
df = df.dropna()

In [0]:
df = df.apply(clean,axis=1)[df.text != 'None']

In [0]:
df = df.drop_duplicates(subset=('text','title'))

In [0]:
if not 'uuid' in df:
    import uuid
    df = df.assign(uuid=tuple(uuid.uuid4() for x in range(len(df))))
    print('done')

done


In [0]:
df.to_pickle('./raw/myNews3.pkl')

# Scrape

In [0]:
config = Config()
config.memoize_articles = False
config.language = 'en'
config.fetch_images = False
config.request_timeout = 60
config.thread_request_timeout = 60
config.follow_meta_refresh = True
config.MIN_WORD_COUNT = 150
config.MIN_SENT_COUNT = 4
root = 'http://www.'
config.browser_user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'

In [0]:
with open('./news_outlets.txt','r') as f:
    outlets = f.read().splitlines()

In [0]:
outlets = [
            'channelnewsasia.com',
            'straitstimes.com',
            'ap.org',
            'reuters.com',
            'todayonline.com',
            'tnp.sg',
            'bbc.com',
            'economist.com',
            'businesstimes.com.sg',
            'businessinsider.sg',
            'nytimes.com',
            'independent.co.uk',
            'express.co.uk',
            'mirror.co.uk',
            'cnbc.com',
            'telegraph.co.uk',
            'washingtontimes.com',
            'washingtonpost.com',
            'nbcnews.com',
            'cnn.com',
            'time.com',
            'latimes.com',
            'nydailynews.com',
            'newsroompost.com',
            'npr.org',
            'nationalgeographic.com',
            'medium.com',
            'discovery.com',
            'sciencedaily.com',
            'phys.org',
            'dailymail.co.uk',
            'cbsnews.com',
            'huffpost.com',
            'abcnews.go.com',
            'dailymail.co.uk',
            'forbes.com',
            'scientificamerican.com',
            'today.com',
            'pbs.org',
            'news.com.au',
            'theatlantic.com',
            'bloomberg.com',
            'foreignaffairs.com',
            'wsj.com',
            'newyorker.com',
            'fortune.com',
            'usatoday.com',
            'asiaone.com',
            'nst.com.my',
            'heraldsun.com.au',
            'nzherald.co.nz'
            ]

In [0]:
papers = {}
for outlet in outlets:
    print(outlet)
    catemap = {}
    paper = news.build(root+outlet,config)
    paper.build()
    papers[outlet] = paper

channelnewsasia.com
straitstimes.com
ap.org
reuters.com
todayonline.com
tnp.sg
bbc.com
economist.com
businesstimes.com.sg
businessinsider.sg
nytimes.com
independent.co.uk
express.co.uk
mirror.co.uk
cnbc.com
telegraph.co.uk
washingtontimes.com
washingtonpost.com
nbcnews.com
cnn.com
time.com
latimes.com
nydailynews.com
newsroompost.com
npr.org
nationalgeographic.com
medium.com
discovery.com
sciencedaily.com
phys.org
dailymail.co.uk
cbsnews.com
huffpost.com
abcnews.go.com
dailymail.co.uk
forbes.com
scientificamerican.com
today.com
pbs.org
news.com.au
theatlantic.com
bloomberg.com
foreignaffairs.com
wsj.com
www.newyorker.com
fortune.com
usatoday.com
asiaone.com
nst.com.my
heraldsun.com.au
www.nzherald.co.nz


In [0]:
news_pool = NewsPool(config)

In [0]:
news_pool.set(papers.values(), threads_per_source=1)

In [0]:
%config Application.log_level="DEBUG"
import datetime,dateutil,json
import pandas as pd
procPapers = pd.DataFrame({},columns=('url','date','outlet','category','title','text','meta_tags','meta'))
not_done = True
while not_done:
    not_done = False
    for outlet,source in papers.items():
        for article in source.articles:
            #resolves articles while threads are still downloading them
            if article.is_parsed: continue
            if article.download_state == 0:
                not_done = True
                continue
            if article.download_state == 1: continue #1 is the enum value of failed
            if article.is_media_news(): continue #if its video/picture based

            article.parse()
            article_dat = {}
            article_dat['outlet'] = outlet
            article_dat['meta_tags'] = set(article.meta_keywords + list(article.tags))
            article_dat['url'] = article.url
            article_dat['meta'] = json.dumps(article.meta_data)
            article_dat['category'] = article.canonical_link
            article_dat['title'] = article.title
            article_dat['text'] = article.text

            if type(article.publish_date) == datetime.datetime:
                article_dat['date'] = article.publish_date.timestamp()
            else:
                try:
                    article_dat['date'] = int(dateutil.parser.parse(article.meta_data.get('pubdate','None')))
                except:
                    article_dat['date'] = -1
            procPapers = procPapers.append(article_dat,ignore_index=True)

            #save ram
            article.html = ''
            article.article_html = ''
            article.doc = None
            article.top_node = None
            article.clean_top_node = None
            article.clean_doc = None
            


#procPapers.to_pickle('./raw/myNews3.pkl')

Building prefix dict from /usr/local/lib/python3.6/dist-packages/jieba/dict.txt ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.2288718223571777 seconds.
Prefix dict has been built succesfully.


In [0]:
df1 = pd.read_pickle('./raw/myNews.pkl')
df2 = pd.read_pickle('./raw/myNews2.pkl')
df3 = pd.read_pickle('./raw/myNews3.pkl')

In [0]:
pd.concat((df1,df2,df3),ignore_index=True,sort=False).drop_duplicates(subset=('text','title')).reset_index(drop=True).to_pickle('./raw/myNewsFinal.pkl')

In [0]:
df.groupby('outlet').count()

,url,date,category,title,text,meta_tags,meta,uuid
outlet,,,,,,,,
abcnews.go.com,98,98,98,98,98,98,98,98
asiaone.com,266,266,266,266,266,266,266,266
bbc.com,90,90,90,90,90,90,90,90
businessinsider.sg,306,306,306,306,306,306,306,306
businesstimes.com.sg,238,238,238,238,238,238,238,238
cbsnews.com,244,244,244,244,244,244,244,244
channelnewsasia.com,40,40,40,40,40,40,40,40
cnbc.com,125,125,125,125,125,125,125,125
cnn.com,743,743,743,743,743,743,743,743


Newspaper's keyword extraction is frequency not tfidf... <br>
Newspaper's summarization is sentence extraction based on frequency of key word....<br>
In conclusion: Use own extractor & paraphraser? or summarizer?<br>
final size: 12k perhaps was ip blocked or smth (evident from the article count from certain sites), create workaround.